In [1]:
import os
import fnmatch

In [2]:
Base_Dir = os.path.join((os.path.expanduser("~")), 'Documents/Visual-Studio/Image-Processing/')
FREESURFER_HOME = '/usr/local/freesurfer/7.4.1-1'
Lut_in = os.path.join(FREESURFER_HOME, 'FreeSurferColorLUT.txt')
Lut_out = os.path.join(os.path.expanduser("~"), '.conda/envs/MRtrix3/share/mrtrix3/labelconvert/fs_default.txt')
Output_Dir = os.path.join(Base_Dir, 'Outputs')
Connection_Dir = os.path.join(Base_Dir, 'Connectivity', 'MR3trix')
os.makedirs(Connection_Dir, 0o755)

In [ ]:
for folder in os.listdir(Output_Dir):
    if folder not in os.listdir(Connection_Dir):
        folder_path = os.path.join(Output_Dir, folder)
        bvec = fnmatch.filter(os.listdir(folder_path), '*.bvec') 
        bval = fnmatch.filter(os.listdir(folder_path), '*.bval')
        dwi = fnmatch.filter(os.listdir(folder_path), '*DTI*.nii')
        if len(bvec) > 0 and len(bval) > 0 and len(dwi) > 0:
            %cd {folder_path}
            bvec = bvec[0]
            bval = bval[0]
            dwi = dwi[0]
            mask = os.path.join(Connection_Dir, folder, 'Mask.mif')
            fod = os.path.join(Connection_Dir, folder, 'FODImg.mif')
            response_text = os.path.join(Connection_Dir, folder, 'responseText.txt')
            track = os.path.join(Connection_Dir, folder, 'Track.tck')
            matrix = os.path.join(Connection_Dir, folder, 'Matrix.csv')
            freesurfer = os.path.join(FREESURFER_HOME, 'subjects', 'sub-' + folder, 'mri', 'aparc+aseg.mgz')
            nodes = os.path.join(Connection_Dir, folder, 'Parcels.nii')
            os.makedirs(os.path.join(Connection_Dir, folder), 0o755)
            !dwi2mask -fslgrad '{bvec}' '{bval}' '{dwi}' '{mask}'
            !dwi2response tournier -fslgrad '{bvec}' '{bval}' '{dwi}' '{response_text}'
            !dwi2fod csd -fslgrad '{bvec}' '{bval}' '{dwi}' '{response_text}' '{fod}' -mask '{mask}'
            !tckgen '{fod}' '{track}' -seed_image '{mask}' -mask '{mask}' -select 5000
            !labelconvert '{freesurfer}' '{Lut_in}' '{Lut_out}' '{nodes}'
            !tck2connectome -symmetric -zero_diagonal -scale_invnodevol '{track}' '{nodes}' '{matrix}'        